In [286]:
import numpy as np
import h5py
import os 
import json
import scipy.io as spio
import matplotlib.pyplot as plt 
import pickle
from matplotlib.mlab import PCA
from scipy.stats.distributions import chi2


# This is a python implementation of 

# Jin et al. (2015), Adaptive reference updating for vibration-based
# structural health monitoring under varying environmental conditions,
# Computers & Structures.

In [147]:
############################################################################
################## STEP2: RUN DAMAGE DETECTION USING APCA ##################
############################################################################
# 
# THIS CODE IS DEVELOPED BASED ON MY WORK IN SHM (2018)
#
# Ref.) 
# Jin et al. (2018),Vibration-based damage detection using 
# online learning algorithm for output-only structural health monitoring.
# Structural Health Monitoring.
#
# Originally coded by SS (2019.08.11) in MATLAB 2018b
# Modified BY SJ CHO (2019.10.15) in MATLAB 2019a
# Converted into Python BY BH Kim (2020.01.23) in Python 3.7.4

In [148]:
## Intialization and Add path required m-files-

####################### OPTIONS #######################

PCA_par={};

# Filename for monitoring data

file_format = '.h5' # or '.h5'

# PCA_par['fn_DB'] = 'DB_case0_lin_1e-05;
# PCA_par['fn_DB'] = 'DB_case1_lin_1e-05;
# PCA_par['fn_DB'] = 'DB_case0_bi_lin_1e-05'
# PCA_par['fn_DB'] = 'DB_case1_bi_lin_1e-05'

# PCA_par['fn_DB'] = 'DB_case0_lin_0.001';
# PCA_par['fn_DB'] = 'DB_case1_lin_0.001';
# PCA_par['fn_DB'] = 'DB_case0_bi_lin_0.001';
PCA_par['fn_DB'] = 'DB_case1_bi_lin_0.001'
PCA_par['fn_DB'] = PCA_par['fn_DB'] + file_format
 

In [149]:
# Siginifiqance Level for Novelty Detection: Threshold
PCA_par['alpha'] = 0.95; # You can change it, but i recommend the values of 95% or 99%

#################### Do not chane the following options #################### 

# Scaling Method  
PCA_par['SD_type'] = 'Z-score' # Do not change it

# Method for selecting number of the retained PCs
PCA_par['nPC_select_algorithm'] = 'eigengap' # Do not change it

# Delayed number for updating PCs to consider disturbance or measurement error
# PCA_par.n_stall=1;
# PCA_par.n_stall=3;
PCA_par['n_stall'] = 10;

In [150]:
if file_format == '.mat':
    op = spio.loadmat(PCA_par['fn_DB'])
    op = op['op']
elif file_format == '.h5':

    with open(PCA_par['fn_DB'], 'rb') as f:  # Python 3: open(..., 'rb')
        op_ = pickle.load(f)

    op = op_.copy()

In [151]:
PCA_par['x'] = np.zeros((2, 2), dtype = np.int16)
PCA_par['x'][0] =[op['IND_x0'][0], op['IND_x0'][-1]]; # INITIAL TRAINIG DATA
PCA_par['x'][1]=[op['IND_x1'][0], op['IND_x1'][-1]]; # TEST DATA
PCA_par['d_indx'] = op['infor']['D_point'] # Sample index at damage ( 0: no damage // 700: damage at # sample index of 700)

In [152]:
t = op['t']; # Assign Temp. to new variable of t
meas = np.hstack((np.transpose(t), op['f'][:,0:2])) # Assign Measurement to new variable of f

## PLOT RAW DATA

# To be updated 
        

In [279]:
a = spio.loadmat('matlab.mat')

In [280]:
X0std = a['X0std']

{'X0std': array([[-1.16119199e+00,  4.80896125e-01,  9.69248396e-01],
        [-1.46947305e+00,  9.46912799e-01,  6.00990706e-01],
        [ 7.65564628e-01, -5.91558956e-01,  1.61236819e-01],
        [ 2.00382688e-01, -9.51852865e-01,  1.77761193e-01],
        [-6.98770399e-01,  6.48361324e-01,  4.61623341e-01],
        [-1.31533252e+00,  7.69390502e-01,  1.35172046e+00],
        [ 1.38212675e+00,  1.40979997e-01, -1.36911653e+00],
        [ 3.03143041e-01, -5.19419280e-01, -8.77209286e-01],
        [-6.73080311e-01,  2.85539258e-01,  9.27210603e-01],
        [-1.18688207e+00,  1.74340293e+00,  5.66707706e-01],
        [ 7.39874540e-01, -8.66518000e-01, -5.26652740e-01],
        [ 6.62804276e-01,  4.19210421e-02, -2.75661331e-01],
        [-1.05843163e+00,  8.47835366e-01, -1.27752747e-01],
        [-1.18688207e+00, -8.00301651e-03,  1.87724407e+00],
        [ 7.39874540e-01, -9.52626753e-01, -1.04730760e+00],
        [ 1.09953577e+00, -9.08816565e-01, -1.81294546e+00],
        [-6.473

In [153]:
#### MAIN LOOP FOR APCA based on Variable Moving Window PCA ####

### DEFINE HYPER-PARAMETERS OF VMWPCA ###

SD_type = PCA_par['SD_type'] # Scaling Method  
alpha = PCA_par['alpha'] # Siginifiqance Level for Novelty Detection: Threshold
nPC_select_algorithm = PCA_par['nPC_select_algorithm'] # Method for selecting number of the retained PCs

x0 = np.arange(PCA_par['x'][0, 0], PCA_par['x'][0, 1]+1) # Initial training samples
x1 = np.arange(PCA_par['x'][1, 0], PCA_par['x'][1, 1]+1) # Test samples

PC = {}
PC['k_cluster'] = np.arange(0,4)  # of clusters for block-wise linearization
PC['n_stall'] = PCA_par['n_stall'] # Delayed number for updating PCs to consider disturbance or measurement error


In [154]:
def Standardization_Data(X0,mu0,sds0,numobs,SD_type):
    import numpy as np 
    
    if SD_type == 'Mean_centered': return (X0 - np.matlib.repmat(mu0,numobs,1));
    elif SD_type == 'Z-score': return np.divide((X0 - np.matlib.repmat(mu0,numobs,1)),  np.matlib.repmat(sds0,numobs,1)) 

In [172]:
def hotelling_tsquared(pc):
    """`pc` should be the object returned by matplotlib.mlab.PCA()."""
    x = pc.a.T
    cov = pc.Wt.T.dot(np.diag(pc.s)).dot(pc.Wt) / (x.shape[1] - 1)
    w = np.linalg.solve(cov, x)
    t2 = (x * w).sum(axis=0)
    return t2

In [284]:
X0

array([[21.5       ,  2.84404796, 11.37720822],
       [20.3       ,  2.84394933, 11.39199257],
       [29.        ,  2.8451878 , 11.36806368],
       [26.8       ,  2.84165711, 11.39461585],
       [23.3       ,  2.84681396, 11.38748285],
       [20.9       ,  2.85044361, 11.41267016],
       [31.4       ,  2.83925065, 11.36667678],
       [27.2       ,  2.8404259 , 11.3781241 ],
       [23.4       ,  2.84220226, 11.39214777],
       [21.4       ,  2.84837396, 11.40166918],
       [28.9       ,  2.8387874 , 11.35550245],
       [28.6       ,  2.84059031, 11.3620764 ],
       [21.9       ,  2.84486348, 11.38219784],
       [21.4       ,  2.85021464, 11.37955801],
       [28.9       ,  2.8445952 , 11.34840904],
       [30.3       ,  2.83850917, 11.37642917],
       [23.5       ,  2.84873709, 11.37652655],
       [21.3       ,  2.84736821, 11.39487654],
       [26.5       ,  2.8416803 , 11.37004444],
       [25.9       ,  2.84219521, 11.378228  ],
       [23.3       ,  2.84430016, 11.392

In [298]:
## STEP #1: Initial PCs based on initial training data

# Training data set
X0 = meas[x0,:]
numobs, numvars = X0.shape
mu0, sds0 = np.mean(X0, 0), np.std(X0, 0)

# Standardization: Z-score (zero-mean and unit variance)

X0std=Standardization_Data(X0,mu0,sds0,numobs,SD_type); 

# PERFORM PCA

pc = PCA(X0std, standardize=False)
loadings = pc.Wt.T
cov = loadings.dot(np.diag(pc.s)).dot(pc.Wt) / (x.shape[1] - 1)
scores = pc.Y
variances = np.sort(np.linalg.eig(cov)[0])[::-1]
tscores = hotelling_tsquared(pc)

# Extracting the retained PC  => Eq. (9) in Ref. (SHM, 2018)
Y = np.max(np.abs(np.diff(variances)))
IX = np.argmax(np.abs(np.diff(variances)))
n_pc = IX




In [301]:
residuals = (X0std - scores[:,0:n_pc+1]*loadings[:,0:n_pc+1].T)
Qdist = np.sqrt(np.sum(np.square(residuals),1));
m_Q=np.mean(Qdist)
V_Q=np.var(Qdist)
V=2*(np.square(m_Q))/V_Q;
distcrit = V_Q/(2*m_Q)*chi2.ppf(alpha,V) #  Threshold limit based on significance level


# Save results of the intial PC model from initial training data

X1 = {}
X1['f0']=X0
X1['mu0']=np.matlib.repmat(mu0,Qdist.shape[0],1)
X1['sds0']=np.matlib.repmat(sds0,Qdist.shape[0],1);

PC['loadings']=loadings 
PC['scores']=scores
PC['n_pc']=n_pc
PC['variances']=variances
PC['tscores']=tscores
PC['update']=1

Q = {}
Q['Qdist']=Qdist
Q['m_Q']=m_Q
Q['V_Q']=V_Q
Q['distcrit']=np.matlib.repmat(distcrit,Qdist.shape[0],1);